# iLykei Lecture Series
# Machine Learning
# Algorithms for Recommenders
# Implementation of libFM in Keras


## Yuri Balasanov, Leonid Nazarov, &copy; iLykei 2019


This notebook shows how to implement Libfm in Keras. The approach has been suggested by [Jean Francois Puget](https://www.ibm.com/developerworks/community/profiles/html/profileView.do?key=9bff030e-4d8e-47a0-a566-8689de958f72#&tabinst=Updates). We use partialy his [blog post](https://www.ibm.com/developerworks/community/blogs/jfp/entry/Implementing_Libfm_in_Keras?lang=en) below.  
Underlying theory is presented on the slides of Lecture 9 **Recommenders**. We recommend also read the seminal paper on Factorization Machines:

Steffen Rendle (2010): <i><a href="https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf">Factorization Machines</a></i>, in Proceedings of the 10th IEEE International Conference on Data Mining (ICDM 2010), Sydney, Australia. 

In [1]:
import pandas as pd
import numpy as np
#import gc

import mlcrate as mlc
import pickle as pkl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import keras
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dot, Reshape, Add, Subtract
from keras import objectives
from keras import backend as K
from keras import regularizers 
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2

from keras.utils import plot_model

Using TensorFlow backend.


The model is a direct implementation of Libfm, with a first embedding layer, then a smart computation of all pairwise dot products.  
The smartness comes direclty from the paper by [Rendle](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf).  

Jean Francois Puget generalized the model to allow for non-categorical input, in which case he uses a dense layer. 

## Problem statement and data

Apply Factorization Machines method to the **movielens-100k** dataset. MovieLens is a benchmark dataset for recommender systems. 

The task is to predict ratings for movies. 

Load the data.

In [2]:
data_movielens = pd.read_csv('u.data', sep='\t',
                   names=['user', 'movie', 'rating', 'timestamp'], header=None)
data_movielens.drop('timestamp',axis=1,inplace=True)
print(data_movielens.shape)
data_movielens.head()

(100000, 3)


,user,movie,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


Before turning to the Movielens-100k look at the following toy example.

In [3]:
data = pd.read_csv('small_example.data', sep='\t',
                   names=['user', 'movie', 'rating', 'timestamp'], header=None)
data.drop('timestamp',axis=1,inplace=True)
print(data.shape)
data

(7, 3)


,user,movie,rating
0,1,1,5
1,1,2,3
2,1,3,1
3,2,3,3
4,2,4,5
5,3,1,1
6,3,3,3


Rating is interpreted as interaction between 'user' and 'movie'.

Calculate upper bound of each feature to prepare for the deep learning model.

In [4]:
features = ['user', 'movie']
f_size  = [int(data[f].max()) + 1 for f in features]
f_size

[4, 5]

Note that the data contain not all combinations of the first two features.

In [5]:
for f in features:
    print(len(data[f].unique()))
f'{len(data.user.unique())*len(data.movie.unique())} > {len(data)}'

3
4


'12 > 7'

Split the data into train and test part. The target column is `rating`.

In [6]:
train,test,y_train,y_test = train_test_split(data, data.rating.values, 
                                    stratify=data.rating.values, test_size=0.3)
X_train = [train[f].values for f in features]
X_test = [test[f].values for f in features]
print(X_train)
print(X_test)

[array([3, 1, 3, 2]), array([1, 1, 3, 3])]
[array([1, 1, 2]), array([2, 3, 4])]


## Embedding

The first step in the analysis is **embedding**, i.e. encoding each input with a vector.

Create a function that takes one level of multilevel categorical variable and encodes it with a level of `k_latent` numbers.

In [7]:
k_latent = 2
embedding_reg = 0.0002 
kernel_reg = 0.1 

Create a matrix of weights to illustrate the work of the embedding function.

In [8]:
weight_matrix = np.arange(12).reshape((-1,2))
print(weight_matrix)

[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]]


In the function embedding layer transforms 1-dimensional categorical varialble `x_input` (for example, user ID or item ID) into `output_dim`-dimensional vector. Then the result gets flattened and returned as tensor.

Arguments of the layer are:

- `input_dim`: size of the vocabulary
- `output_dim`: dimension of the vector space in which words are embedded
- `input_length`: length of input sequences.


In [9]:
out_dim=2
def get_embed(x_input, x_size, out_dim,test_weights=False):
    # x_input is index of input (either user or item)
    # x_size is length of vocabulary (e.g. total number of users or items)
    # test_weights is a demo flag to show results with predefined weights
    # out_dim is size of embedding vectors
    if x_size > 0: #category
        if test_weights & (out_dim<=2):
            embed = Embedding(x_size, out_dim, input_length=1,
                          weights=[weight_matrix[:x_size,:out_dim]], 
                          embeddings_regularizer=l2(embedding_reg))(x_input)
        else:
            embed = Embedding(x_size, out_dim, input_length=1,
                              embeddings_regularizer=l2(embedding_reg))(x_input)
        embed = Flatten()(embed)
    else:
        embed = Dense(out_dim, kernel_regularizer=l2(embedding_reg))(x_input)
    return embed

To illustrate working of the function create a list of inputs `X` and make a model with the output tensor generated by `get_embed()` with predefined weights.

In [10]:
X = [2,3,2]
inp = Input(shape=(1,))
emd_model = Model(inp,get_embed(inp,max(X)+1,out_dim,test_weights=True))
plot_model(emd_model, to_file='emb_model.png',
           show_shapes=True,show_layer_names=True)
emd_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1, 2)              8         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2)                 0         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


![Embedding Model](emb_model.png)

In [11]:
emd_model.predict(X)

array([[ 4.,  5.],
       [ 6.,  7.],
       [ 4.,  5.]], dtype=float32)

Prediction is the list of rows of `weight_matrix` corresponding to values of `X`.

Note that `X[0]` and `X[2]` are equal and generated identical vectors.

To achieve the same result for non-categorical input set `x_size=0`:

In [12]:
Model(inp,get_embed(inp,0,out_dim)).predict(np.sqrt(X))

array([[ 0.63004494,  0.36816451],
       [ 0.77164435,  0.45090762],
       [ 0.63004494,  0.36816451]], dtype=float32)

## LibFM model implementation in Keras

### Some useful Keras layers and objects

This section explains some layers and objects used in the LibFM implementation in the following section.

#### Zip object

Implementation code of LibFM below uses `zip` object which makes an iterator that aggregates elements from each of the iterables and returns an iterator of tuples, where the i-th tuple contains the i-th element from each of the argument sequences or iterables. Elements are loaded into memory on demand.

In [13]:
print('Range as list: ',list(range(3)))
print('String as list: ',list('abcd'))
print('Zip object as list: ',list(zip(range(3), 'abcd')))

Range as list:  [0, 1, 2]
String as list:  ['a', 'b', 'c', 'd']
Zip object as list:  [(0, 'a'), (1, 'b'), (2, 'c')]


#### Layer Add

Layer `Add` takes as input a list of tensors,
all of the same shape, adds them and returns
a single tensor (also of the same shape).

In [14]:
input1 = keras.layers.Input(shape=(16,))
x1 = keras.layers.Dense(8, activation='relu')(input1)
input2 = keras.layers.Input(shape=(32,))
x2 = keras.layers.Dense(8, activation='relu')(input2)
added = keras.layers.Add()([x1, x2])

out = keras.layers.Dense(4)(added)
model = keras.models.Model(inputs=[input1, input2], outputs=out)
plot_model(model, to_file='modelAdd.png',show_shapes=True,
           show_layer_names=True)

![Model Add](modelAdd.png)

#### Layer Subtract

Layer `Subtract` takes as input a list of tensors of size 2, both of the same shape, and returns a single tensor, (inputs[0] - inputs[1]), also of the same shape.

In [15]:
input1 = keras.layers.Input(shape=(16,))
x1 = keras.layers.Dense(8, activation='relu')(input1)
input2 = keras.layers.Input(shape=(32,))
x2 = keras.layers.Dense(8, activation='relu')(input2)
# Equivalent to subtracted = keras.layers.subtract([x1, x2])
subtracted = keras.layers.Subtract()([x1, x2])

out = keras.layers.Dense(4)(subtracted)
model = keras.models.Model(inputs=[input1, input2], outputs=out)
plot_model(model, to_file='modelSubtract.png',show_shapes=True,
           show_layer_names=True)

![Model Subtract](modelSubtract.png)

#### Layer Dot

Layer `Dot` computes a dot product between samples in two tensors.

E.g. if applied to a list of two tensors a and b of shape (batch_size, n), the output will be a tensor of shape (batch_size, 1).

Arguments

- axes: Integer or tuple of integers, axis or axes along which to take the dot product.
- normalize: Whether to L2-normalize samples along the dot product axis before taking the dot product. If set to True, then the output of the dot product is the cosine proximity between the two samples.


### Steps of LibFM

#### Model

Follow all steps of LibFM on a simple example.

The general LibFM model is a linear model with interactions:

$$\hat{y}_{j,h} = w_0+\sum_{j=1}^{n_u} w_j^{(u)}x_j^{(u)} +\sum_{h=1}^{n_i}w_h^{(i)}x_h^{(i)} + \sum_{j}\sum_{h} w_{j,h} x_j^{(u)}x_h^{(i)},$$
where: 

- $\hat{y}_{j,h}$ is predicted response, i.e. rating of item $h$ by user $j$
- $x_j^{(u)}$ is the dummy variable for user $j$;
- $x_h^{(i)}$ is the dummy variable for item $h$;
- $w_j^{(u)}$ is the main effect of user $j$;
- $w_h^{(i)}$ is the main effect of item $h$;
- $w_{j,h}$ is the interaction between user $j$ and item $h$.

Factorization with $k$ factors in the formula above applies to the matrix of interactions $\{ w_{j,h} \}$:
$$w_{j,h}=\sum_{g=1}^k v_{j,g}^{(u)}v_{h,g}^{(i)},$$
where 
$$v_j^{(u)}=(v_{j,1}^{(u)},\ldots,v_{j,k}^{(u))})$$ 
is the vector of factors for user $j$ and 
$$v_h^{(i)}=(v_{h,1}^{(i)},\ldots,v_{h,k}^{(i)})$$ 
is the vector of factors for item $h$.

In case of one active user $j_{u_a}$ and one active item $h_{j_a}$ the model simplifies to
$$\hat{y}_{j_{u_a},h_{i_a}} = w_0+w_{j_{u_a}}^{(u)}+w_{h_{i_a}}^{(i)} +  w_{j_{u_a},h_{i_a}},$$
where
$$w_{j_{u_a},h_{i_a}}=\sum_{g=1}^k v_{j_{u_a},g}^{(u)}v_{h_{i_a},g}^{(i)}.$$

#### Computation of interactions

It is possible to organize computation of interaction terms in a very efficient way and reduce complexity of computation from $O(kn^2)$ to only $O(kn)$.

The idea is the following: instead of calculation  
$$F = \sum _{j<h} (f_j \cdot f_h),$$ 
which has complexity $O(kn^2)$, calculate 
$$F =\frac{1}{2} \sum_{j} (f_j \cdot (S - f_j)),$$
where $(a \cdot b)$ is dot product of vectors $a$ and $b$ and $S=\sum_j f_j$.
Alternative calculation requires only $n$ dot products and has complexity $O(kn)$. 

Lemma in the [paper](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf) applies the alternative calculation for $f_j=v_{j}^{(u)}x_j^{(u)},f_h=v_{h}^{(i)}x_h^{(i)}$

In particular:
$$\sum_{j}\sum_{h} \left(v_{j}^{(u)} \cdot v_{h}^{(i)}\right) x_j^{(u)}x_h^{(i)}=\sum_{j}\sum_{h}\left( \sum_{g=1}^k \left(v_{j,g}^{(u)}x_j^{(u)}\right) \left(v_{h,g}^{(i)}x_h^{(i)} \right) \right)$$
$$=\sum_{j}\sum_{h}\left( \left(v_{j,g}^{(u)}x_j^{(u)}\right) \cdot \left(v_{h,g}^{(i)}x_h^{(i)} \right) \right) $$
So, calculation of interactions is done with dot products of factor vectors $f_j$ and vectors $S=\sum_g f_g-f_j$. 

### 1. Input layer

Create input as list of 2 tensors representing index of user $u_a$ and index of item $i_a$. Each tensor has shape `(None,1)`

In [16]:
dim_input = len(f_size)
    
input_x = [Input(shape=(1,)) for i in range(dim_input)]
input_x

[<tf.Tensor 'input_6:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'input_7:0' shape=(?, 1) dtype=float32>]

### 2. Main effects

Create a list of tensors for linear terms $\sum_{j=1}^{n_u} w_j^{(u)}x_j^{(u)}, \sum_{h=1}^{n_i}w_h^{(i)}x_h^{(i)}$.

In [17]:
lin_terms = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]
lin_terms

[<tf.Tensor 'flatten_2/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_3/Reshape:0' shape=(?, ?) dtype=float32>]

The result is list of terms 
$$ \left[ w_1^{(u)}x_1^{(u)},\ldots,w_{n_u}^{(u)}x_{n_u}^{(u)},w_1^{(i)}x_1^{(i)},\ldots,w_{n_i}^{(i)}x_{n_i}^{(i)} \right] .$$

In case of collaborative filtering with dummy variables for inputs the list reduces to $\left[ w_{j_{u_a}}^{(u)},w_{h_{i_a}}^{(i)} \right].$

Number of trained coefficients in this layer is $n_u+n_i$.

In [18]:
print(f_size)

[4, 5]


In [19]:
input_x = [Input(shape=(1,)) for i in range(dim_input)] 
    
lin_terms = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]
model = Model(inputs=input_x, outputs=lin_terms)
plot_model(model, to_file='modelLinterms.png',show_shapes=True,
           show_layer_names=True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 1)         4           input_8[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 1)         5           input_9[0][0]                    
__________________________________________________________________________________________________
flatten_4 

![Model Linterms](modelLinterms.png)

Check predicted linear terms with randomly initiated weights.

In [20]:
lin_terms=model.predict(X_train)

For the user input of `X_train`:

In [21]:
lin_terms[0]

array([[ 0.03469365],
       [-0.02713688],
       [ 0.03469365],
       [ 0.01670531]], dtype=float32)

Print the user input of `X_train`, user weights of the model and reconstruct predictions of linear terms for user using this information:

In [22]:
print('User index:\n',X_train[0])
print('User weights:\n',np.concatenate(model.layers[2].get_weights()[0])) #list of lists to list
print('Reconstructed user linear terms:')
for User in X_train[0]: print(np.concatenate(model.layers[2].get_weights()[0])[User])

User index:
 [3 1 3 2]
User weights:
 [ 0.01672569 -0.02713688  0.01670531  0.03469365]
Reconstructed user linear terms:
0.0346936
-0.0271369
0.0346936
0.0167053


The cell above shows that prediction is the weight of the model indexed by the value of the input.

Same way reconstruct predictions of the linear terms for item. Note that item weights are in layer 3, the second embedding layer of the model.

In [23]:
print(lin_terms[1])
print(X_train[1])
print(np.concatenate(model.layers[3].get_weights()[0])) #list of lists to list
print('Reconstructed item linear terms:')
for User in X_train[1]: print(np.concatenate(model.layers[3].get_weights()[0])[User])

[[ 0.0396462 ]
 [ 0.0396462 ]
 [-0.00811861]
 [-0.00811861]]
[1 1 3 3]
[-0.0155157   0.0396462  -0.0332055  -0.00811861 -0.04376755]
Reconstructed item linear terms:
0.0396462
0.0396462
-0.00811861
-0.00811861


### 3. Factors

Creating factors $v_j x_j$ for users and items is similar to creating main effects, but dimension of embedding space is equal to the selected number of factors `k_latent`.

In [24]:
k_latent

2

In [25]:
factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
factors

[<tf.Tensor 'flatten_6/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_7/Reshape:0' shape=(?, ?) dtype=float32>]

Output of this layer is list of 2 vectors:
$$ \left[ v_j^{(u)},v_h^{(i)} \right]= \left[ (v_{j,1}^{(u)},\ldots,v_{j,k}^{(u)}),(v_{h,1}^{(i)},\ldots,v_{h,k}^{(i)}) \right],$$
where $k$ is the selected number of factors `k_latent`.

In the example below $k=2$.

In [26]:
input_x = [Input(shape=(1,)) for i in range(dim_input)] 
    
factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
model = Model(inputs=input_x, outputs=factors)
plot_model(model, to_file='modelFactors.png',show_shapes=True,
           show_layer_names=True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1, 2)         8           input_10[0][0]                   
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 1, 2)         10          input_11[0][0]                   
__________________________________________________________________________________________________
flatten_8 

![Model Factors](modelFactors.png)

Predict factors using the model and reconstruct them with randomly initiated weights.

In [27]:
factrs=model.predict(X_train)

For user factors:

In [28]:
print('User index:\n',X_train[0])
print('User weights:\n',np.concatenate(model.layers[2].get_weights()[0]))
print('Predicted user factors:\n',factrs[0])
print('Reconstructed user factors:')
for User in X_train[0]: print(np.concatenate(model.layers[2].get_weights()[0])[2*User],
                             np.concatenate(model.layers[2].get_weights()[0])[2*User+1])

User index:
 [3 1 3 2]
User weights:
 [-0.00405724 -0.04468237  0.01256463  0.0371066   0.00792453 -0.0283061
  0.04394001 -0.03884629]
Predicted user factors:
 [[ 0.04394001 -0.03884629]
 [ 0.01256463  0.0371066 ]
 [ 0.04394001 -0.03884629]
 [ 0.00792453 -0.0283061 ]]
Reconstructed user factors:
0.04394 -0.0388463
0.0125646 0.0371066
0.04394 -0.0388463
0.00792453 -0.0283061


For item factors: 

In [29]:
print('User index:\n',X_train[0])
print('User weights:\n',np.concatenate(model.layers[2].get_weights()[0]))
print('Predicted item factors:\n',factrs[1])
print('Reconstructed item factors:')
for User in X_train[1]: print(np.concatenate(model.layers[3].get_weights()[0])[2*User],
                             np.concatenate(model.layers[3].get_weights()[0])[2*User+1])

User index:
 [3 1 3 2]
User weights:
 [-0.00405724 -0.04468237  0.01256463  0.0371066   0.00792453 -0.0283061
  0.04394001 -0.03884629]
Predicted item factors:
 [[ 0.02250214  0.02847942]
 [ 0.02250214  0.02847942]
 [-0.04329686 -0.02746215]
 [-0.04329686 -0.02746215]]
Reconstructed item factors:
0.0225021 0.0284794
0.0225021 0.0284794
-0.0432969 -0.0274621
-0.0432969 -0.0274621


Again, factors for both users and items are the weights of the model indexed by the input values, but dimension of each factor is now `k_latent`.

### 4. Adding factors

Addition of factors is necessary to calculate $S=\sum_g f_g$ involved in calculation of interactions as part of $\left(S-f_j\right)$. 

In case of collaborative filtering the result becomes:
$$ \left[ v_j^{(u)}+v_h^{(i)} \right]= \left[ v_{j,1}^{(u)}+v_{h,1}^{(i)},\ldots,v_{j,k}^{(u)}+v_{h,k}^{(i)} \right]$$

In [30]:
s = Add()(factors)
s

<tf.Tensor 'add_2/add:0' shape=(?, ?) dtype=float32>

In [31]:
inp = [Input(shape=(2,)),Input(shape=(2,))]
add = Model(inp,Add()(inp))
plot_model(add, to_file='modelAdd.png',show_shapes=True,
           show_layer_names=True)
add.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
add_3 (Add)                     (None, 2)            0           input_12[0][0]                   
                                                                 input_13[0][0]                   
Total params: 0
Trainable params: 0
Non-trainable params: 0
__________________________________________________________________________________________________


![Model Add](modelAdd.png)

The following cell shows that output of this layer is equal to:

- First column is first user factor, plus first item factor
- Second column is second user factor, plus second item factor 

In [32]:
s_pred=add.predict(factrs)
print('User factors:\n',factrs[0])
print('\nItem factors:\n',factrs[1])
print('\nAdded columns of factors for user and for item:\n',np.sum(factrs,axis=0))
print('\nResult of Add layer:\n',s_pred)

User factors:
 [[ 0.04394001 -0.03884629]
 [ 0.01256463  0.0371066 ]
 [ 0.04394001 -0.03884629]
 [ 0.00792453 -0.0283061 ]]

Item factors:
 [[ 0.02250214  0.02847942]
 [ 0.02250214  0.02847942]
 [-0.04329686 -0.02746215]
 [-0.04329686 -0.02746215]]

Added columns of factors for user and for item:
 [[ 0.06644215 -0.01036687]
 [ 0.03506676  0.06558602]
 [ 0.00064315 -0.06630844]
 [-0.03537233 -0.05576825]]

Result of Add layer:
 [[ 0.06644215 -0.01036687]
 [ 0.03506676  0.06558602]
 [ 0.00064315 -0.06630844]
 [-0.03537233 -0.05576825]]


### 5. Calculation of term $\left(S-f_j\right)$

To calculate difference $\left(S-f_j\right)$ use `Subtract` layer with precalculated tensors for $S$ and $f_j$.

In [33]:
diffs = [Subtract()([s, x]) for x in factors]
diffs

[<tf.Tensor 'subtract_2/sub:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'subtract_3/sub:0' shape=(?, ?) dtype=float32>]

In [34]:
s = Input(shape=(2,))
fact = [Input(shape=(2,)),Input(shape=(2,))]
diffs = Model([s]+fact,[Subtract()([s, x]) for x in fact])
plot_model(diffs, to_file='modelDiff.png',show_shapes=True,
           show_layer_names=True)
diffs.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
subtract_4 (Subtract)           (None, 2)            0           input_14[0][0]                   
                                                                 input_15[0][0]                   
__________

![Model Diff](modelDiff.png)

The middle input tensor is $S$ calculated above by the Add layer, the other two inputs are user factors and item factors. The output is a list of 2 tensors: one for user and one for factor.

Output for user is calculated as difference between matrix of sums `s_pred` and the user factor matrix. Correspondingly, difference between `s_pred` and item factor matrix is the output for item.

In [35]:
print('User factors:\n',factrs[0],'\nItem factors:\n',factrs[1])
print('Sum of factors:\n',s_pred)
dffs=diffs.predict([s_pred,factrs[0],factrs[1]])
print('Output of layer Subtract for user:\n',dffs[0],
     '\nOutput of layer Subtract for item:\n',dffs[1])
print('Reconstructed layer Subtract for user:\n',np.subtract(s_pred,factrs[0]),
     '\nReconstructed layer Subtract for item:\n',np.subtract(s_pred,factrs[1]))

User factors:
 [[ 0.04394001 -0.03884629]
 [ 0.01256463  0.0371066 ]
 [ 0.04394001 -0.03884629]
 [ 0.00792453 -0.0283061 ]] 
Item factors:
 [[ 0.02250214  0.02847942]
 [ 0.02250214  0.02847942]
 [-0.04329686 -0.02746215]
 [-0.04329686 -0.02746215]]
Sum of factors:
 [[ 0.06644215 -0.01036687]
 [ 0.03506676  0.06558602]
 [ 0.00064315 -0.06630844]
 [-0.03537233 -0.05576825]]
Output of layer Subtract for user:
 [[ 0.02250214  0.02847942]
 [ 0.02250214  0.02847942]
 [-0.04329686 -0.02746215]
 [-0.04329686 -0.02746215]] 
Output of layer Subtract for item:
 [[ 0.04394001 -0.03884629]
 [ 0.01256463  0.0371066 ]
 [ 0.04394001 -0.03884629]
 [ 0.00792453 -0.0283061 ]]
Reconstructed layer Subtract for user:
 [[ 0.02250214  0.02847942]
 [ 0.02250214  0.02847942]
 [-0.04329686 -0.02746215]
 [-0.04329686 -0.02746215]] 
Reconstructed layer Subtract for item:
 [[ 0.04394001 -0.03884629]
 [ 0.01256463  0.0371066 ]
 [ 0.04394001 -0.03884629]
 [ 0.00792453 -0.0283061 ]]


### 6. Dot products

Finally, everything is ready for calculation of the key expression of the model:
$$\sum_{j} (f_j \cdot (S - f_j)),$$
which is dot product of factors and the output of layer `Subtract`.

In [36]:
dif = [Input(shape=(2,)),Input(shape=(2,))]
fact = [Input(shape=(2,)),Input(shape=(2,))]
dots = Model(dif+fact,
            [Dot(axes=1)([d, x]) for d,x in zip(dif, fact)])
plot_model(dots, to_file='modelDot.png',show_shapes=True,
           show_layer_names=True)
dots.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
dot_1 (Dot

![Model Dot](modelDot.png)

The left two input tensors are differences $(S - f_j)$ and factors for user and the right two input tensors are differences and factors for items.

The output of the layer is list of dot product for user and for item.

In [37]:
dots_res = dots.predict([dffs[0],dffs[1],factrs[0],factrs[1]])
print('Diffs for user:\n',dffs[0],'\nDiffs for item:\n',dffs[1])
print('User factors:\n',factrs[0],'\nItem factors:\n',factrs[1])
print('Output for users:\n',dots_res[0],'\nOutput for items:\n',dots_res[1])
print(np.diagonal(np.dot(dffs[0],factrs[0].transpose())))
print(np.diagonal(np.dot(dffs[1],factrs[1].transpose())))

Diffs for user:
 [[ 0.02250214  0.02847942]
 [ 0.02250214  0.02847942]
 [-0.04329686 -0.02746215]
 [-0.04329686 -0.02746215]] 
Diffs for item:
 [[ 0.04394001 -0.03884629]
 [ 0.01256463  0.0371066 ]
 [ 0.04394001 -0.03884629]
 [ 0.00792453 -0.0283061 ]]
User factors:
 [[ 0.04394001 -0.03884629]
 [ 0.01256463  0.0371066 ]
 [ 0.04394001 -0.03884629]
 [ 0.00792453 -0.0283061 ]] 
Item factors:
 [[ 0.02250214  0.02847942]
 [ 0.02250214  0.02847942]
 [-0.04329686 -0.02746215]
 [-0.04329686 -0.02746215]]
Output for users:
 [[-0.00011758]
 [ 0.00133951]
 [-0.00083566]
 [ 0.00043424]] 
Output for items:
 [[-0.00011758]
 [ 0.00133951]
 [-0.00083566]
 [ 0.00043424]]
[-0.00011758  0.00133951 -0.00083566  0.00043424]
[-0.00011758  0.00133951 -0.00083566  0.00043424]


The final steps of the implementation are:

- Concatenating linear terms and dots in order to put all terms of the model together:
$$\hat{y}_{j,h} = w_0+\sum_{j=1}^{n_u} w_j^{(u)}x_j^{(u)} +\sum_{h=1}^{n_i}w_h^{(i)}x_h^{(i)} + \sum_{j}\sum_{h} \left(\sum_{g=1}^k v_{j,g}^{(u)}v_{h,g}^{(i)}\right) x_j^{(u)}x_h^{(i)},$$
where the first 2 sums are linear terms and the all vectors of the last expression are returned by the `Dot` layer.
- Adding batch normalization for stability
- Adding the output layer with 1 unit and "ReLU" activation for positive rating prediction

Note that in the final layer all terms will get their weights and will be added together to make the model formula.

### Putting it all together

The following function puts all steps of LibFM algorithm togehter.

In [38]:
def build_model(f_size):
    dim_input = len(f_size)
    
    input_x = [Input(shape=(1,)) for i in range(dim_input)] 
    
    lin_terms = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]

    factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
     
    s = Add()(factors)
    
    diffs = [Subtract()([s, x]) for x in factors]
    
    dots = [Dot(axes=1)([d, x]) for d,x in zip(diffs, factors)]
    
    x = Concatenate()(lin_terms + dots)
    x = BatchNormalization()(x)
    output = Dense(1, activation='relu', kernel_regularizer=l2(kernel_reg))(x)
    model = Model(inputs=input_x, outputs=[output])
    model.compile(optimizer=Adam(clipnorm=0.25), loss='mean_squared_error')
    return model

The author uses parameter `clipnorm=0.5` in the Adam optimizer. This parameter is common for all Keras optimizers and enables avoiding so called "Exploding Gradients". Gradients will be clipped when their L2 norm exceeds clipnorm value.  

It is recommended to tune this parameter to obtain better results.

Check the complete model summary and plot its diagram.

In [39]:
model = build_model(f_size)
plot_model(model, to_file='modelComplete.png',show_shapes=True,
           show_layer_names=True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 1, 2)         8           input_21[0][0]                   
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 1, 2)         10          input_22[0][0]                   
__________________________________________________________________________________________________
flatten_12

![Final model](modelComplete.png)

Prepare the Movielens-100k data

In [40]:
features = ['user', 'movie']
f_size  = [int(data_movielens[f].max()) + 1 for f in features]
f_size

[944, 1683]

In [41]:
train,test,y_train,y_test = train_test_split(data_movielens, data_movielens.rating.values, 
                                    stratify=data_movielens.rating.values, test_size=0.3)
X_train = [train[f].values for f in features]
X_test = [test[f].values for f in features]
print(X_train)
print(X_test)

[array([ 12, 780, 693, ...,  56, 548, 731]), array([ 238,  485,  215, ...,   11,  323, 1503])]
[array([749, 539, 503, ..., 121, 401, 577]), array([ 23, 357, 729, ..., 515, 211,  31])]


We can fit the model to train data. We stop training as soon as validation loss does not improve during 2 epochs.

In [42]:
n_epochs = 100

batch_size = 128
print('Batch size: ',batch_size)
model = build_model(f_size)
earlystopper = EarlyStopping(patience=2, verbose=1)
t = mlc.time.Timer()
model.fit(X_train,  y_train, 
          epochs=n_epochs, batch_size=batch_size, verbose=0, shuffle=True, 
          validation_data=(X_test, y_test), 
          callbacks=[earlystopper],
         )
print('\n')
print(t.fsince(0))
print('RMSE',model.evaluate(X_test, y_test,
                            batch_size=batch_size,verbose=0)**0.5)

Batch size:  128
Epoch 00025: early stopping


1m04s
RMSE 0.956454732649


Alex Rogozhnikov compared different implementations of Libfm in his post [Testing implementations of LibFM](http://arogozhnikov.github.io/2016/02/15/TestingLibFM.html) and got the following table for sklearn LogisticRegression, original Libfm and two python packages fastFM and pylibfm.


|package   | time  |RMSE   |
|---|---|---|
|  logistic | 0.059469  | 0.942771  |
| libFM  |  8.970990 |  0.913520 |
| fastFM  |  4.840041 | 0.915184  |
| pylibfm  |  13.157164 | 0.944870  |

The fit of the model in this notebook is in line with the reported results. 

However, the model can be improved by further tuning the model hyperparameters and by averaging predictions after several runs.